In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from keras import Sequential
from keras.layers import Dense, Activation
from numpy.random import normal as norm, uniform as uni
from keras.losses import categorical_crossentropy
from sklearn.metrics import log_loss
from copy import deepcopy
from time import time

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

# Task 1. Iris dataset

In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

enc = OneHotEncoder()
Y = enc.fit_transform(y[:, np.newaxis]).toarray()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.2, random_state=2)

n_features = X.shape[1]
n_classes = Y.shape[1]

In [3]:
def my_model():
    
    model = Sequential()
    
    model.add(Dense(units=6, input_dim=n_features, use_bias=True,activation='tanh'))
    
    model.add(Dense(units=6, use_bias=True, activation='tanh'))
    
    model.add(Dense(units=n_classes, use_bias=True, activation='softmax'))
    
    return model

In [4]:
def sample_weights(model, std=0.5):
    
    def sample_array_from_array(array):
        new_array = []
        for value in array:
            new_array += [norm(value, std, 1)[0]]
        
        return new_array
    model = deepcopy(model)
    for idx, layer in enumerate(model.layers):
        weights = layer.get_weights()
        W, bias = weights
        new_W = []
        for element in W:
            new_W.append(sample_array_from_array(element))
        new_bias = sample_array_from_array(bias)
        model.layers[idx].set_weights([np.asarray(new_W), np.asarray(new_bias)])

    return model  

In [5]:
def SA(model, dataset, a, min_t):
    
    def calculate_p(model, dataset, T):
        
        return np.exp(-get_loss(model,dataset)/T)
    
    def calculate_new_T(T,a):
        
        return T*a
    
    def get_loss(model, dataset):
        
        x, y_true = dataset
        y_pred = model.predict(x)
        
        return log_loss(y_true, y_pred)
    
    
    
    loss = get_loss(model, dataset)
    T = 1.5
    steps = 0
    accepted_model = deepcopy(model)
    best_model = deepcopy(model)
    
    
    while (T > min_t):
        
        proposed_model = sample_weights(accepted_model)
        
        proposed_p, accepted_p = calculate_p(proposed_model,dataset,T), calculate_p(accepted_model,dataset,T)
        
        if proposed_p > accepted_p:
            
            accepted_model = proposed_model
            
            if get_loss(accepted_model,dataset) < get_loss(best_model, dataset):
                best_model = accepted_model

        else:
        
            U = uni(0,1)
            alpha = proposed_p/accepted_p

            if U <= alpha:
                accepted_model = proposed_model
        
        T = calculate_new_T(T,a)
        
        steps += 1
    
    return best_model

In [6]:
model = my_model()

W0130 22:53:50.641494 140734765067712 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0130 22:53:50.655380 140734765067712 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0130 22:53:50.658118 140734765067712 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [7]:
start = time()
best_model = SA(model,[X_train,Y_train], 0.9,0.5)
end = time()
print('Time taken: {} seconds'.format(end - start))

W0130 22:53:51.301769 140734765067712 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0130 22:53:51.303787 140734765067712 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Time taken: 1.7999250888824463 seconds


In [8]:
y_pr = best_model.predict(X_test)
from sklearn.metrics import r2_score
r2_score(Y_test, y_pr)

0.35269168757311276

In [9]:
backprob_model = my_model()
backprob_model.compile(optimizer='adam',loss=categorical_crossentropy)
start = time()
backprob_model.fit(x=X_train,y=Y_train,epochs=200)
end = time()
print('Time taken: {} seconds'.format(end - start))

W0130 22:53:54.647950 140734765067712 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0130 22:53:54.740096 140734765067712 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/200
120/120 [==============================] - 0s 2ms/step - loss: 1.1813
Epoch 2/200
120/120 [==============================] - 0s 39us/step - loss: 1.1555
Epoch 3/200
120/120 [==============================] - 0s 38us/step - loss: 1.1303
Epoch 4/200
120/120 [==============================] - 0s 49us/step - loss: 1.1061
Epoch 5/200
120/120 [==============================] - 0s 39us/step - loss: 1.0824
Epoch 6/200
120/120 [==============================] - 0s 47us/step - loss: 1.0592
Epoch 7/200
120/120 [==============================] - 0s 42us/step - loss: 1.0362
Epoch 8/200
120/120 [==============================] - 0s 43us/step - loss: 1.0143
Epoch 9/200
120/120 [==============================] - 0s 44us/step - loss: 0.9920
Epoch 10/200
120/120 [==============================] - 0s 40us/step - loss: 0.9701
Epoch 11/200
120/120 [==============================] - 0s 45us/step - loss: 0.9488
Epoch 12/200
120/120 [==============================] - 0s 42us/step - loss: 0.9287
Ep

120/120 [==============================] - 0s 47us/step - loss: 0.3356
Epoch 99/200
120/120 [==============================] - 0s 39us/step - loss: 0.3329
Epoch 100/200
120/120 [==============================] - 0s 44us/step - loss: 0.3304
Epoch 101/200
120/120 [==============================] - 0s 44us/step - loss: 0.3277
Epoch 102/200
120/120 [==============================] - 0s 40us/step - loss: 0.3250
Epoch 103/200
120/120 [==============================] - 0s 44us/step - loss: 0.3225
Epoch 104/200
120/120 [==============================] - 0s 44us/step - loss: 0.3199
Epoch 105/200
120/120 [==============================] - 0s 39us/step - loss: 0.3173
Epoch 106/200
120/120 [==============================] - 0s 45us/step - loss: 0.3147
Epoch 107/200
120/120 [==============================] - 0s 39us/step - loss: 0.3120
Epoch 108/200
120/120 [==============================] - 0s 35us/step - loss: 0.3094
Epoch 109/200
120/120 [==============================] - 0s 40us/step - loss: 0.

120/120 [==============================] - 0s 35us/step - loss: 0.1429
Epoch 195/200
120/120 [==============================] - 0s 39us/step - loss: 0.1419
Epoch 196/200
120/120 [==============================] - 0s 34us/step - loss: 0.1407
Epoch 197/200
120/120 [==============================] - 0s 36us/step - loss: 0.1396
Epoch 198/200
120/120 [==============================] - 0s 39us/step - loss: 0.1385
Epoch 199/200
120/120 [==============================] - 0s 37us/step - loss: 0.1374
Epoch 200/200
120/120 [==============================] - 0s 40us/step - loss: 0.1365
Time taken: 1.584826946258545 seconds


In [10]:
bkpr_y_pr = backprob_model.predict(X_test)
r2_score(Y_test,bkpr_y_pr)

0.921252614727876

# Task2. Combinatorial optimization

In [11]:
import pandas as pd
import math
from random import randrange

In [12]:
cities = pd.read_csv('./city.csv')
cities.population = cities.population.astype(str).str.replace('\[', '').str.replace('\]', '').astype(int)
top_30_cities = cities.sort_values('population',ascending=False).head(30)

In [13]:
city_name, city_lat, city_lon = [], [], []
for index, data in top_30_cities.iterrows():
    city_name.append(data.address)
    city_lat.append(data.geo_lat)
    city_lon.append(data.geo_lat)

In [14]:
cities_info = [city_lat, city_lon, city_name]

In [15]:
def city_SA(cities_info, a, min_t):
    
    def calculate_p(proposed_path, T):
        
        return np.exp(-get_loss(proposed_path)/T)
    
    def calculate_new_T(T,a):
        
        return T*a
    
    def get_loss(proposed_path):
        
        cities_lat, cities_lon, cities_name = proposed_path
        
        return calculate_path_distance(cities_lat,cities_lon)
    
    def swap_city(idx1, idx2, lat_array, lon_array, city_name):
        
        lat_temp, lon_temp, city_temp = lat_array[idx1], lon_array[idx1], city_name[idx1]
        lat_array[idx1] = lat_array[idx2]
        lon_array[idx1] = lon_array[idx2]
        city_name[idx1] = city_name[idx2]
        lat_array[idx2], lon_array[idx2], city_name[idx2] = lat_temp, lon_temp, city_temp

        return lat_array, lon_array, city_name
    
    def calculate_distance(lat1,lon1,lat2,lon2):
    #https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula
        def degres_to_radians(degrees):

            return degrees * (math.pi/180)

        R = 6371
        lat = degres_to_radians(lat2-lat1)
        lon = degres_to_radians(lon2-lon1)
        a = math.sin(lat/2) * math.sin(lat/2) + math.cos(degres_to_radians(lat1))\
        * math.cos(degres_to_radians(lat2)) * math.sin(lon/2) * math.sin(lon/2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distance = R * c

        return distance
    
    def calculate_path_distance(lat_array, lon_array):
        
        pairs = []
        for i in range(len(lat_array)-1):
            pairs.append([i,i+1])
        pairs.append([29,0])

        total_distance = 0

        for pair in pairs:
            a , b = pair
            lat_a, lon_a = lat_array[a], lon_array[a]
            lat_b, lon_b = lat_array[b], lon_array[b]
            ab_distance = calculate_distance(lat_a,lon_a,lat_b,lon_b)
            total_distance += ab_distance

        return total_distance
    
    
    cities_lat, cities_lon, cities_name = cities_info
    cities_lat, cities_lon, cities_name = swap_city(randrange(0,29), randrange(0,29), cities_lat, cities_lon, cities_name)
    
    loss = get_loss(cities_info)
    T = 1000
    steps = 0
    accepted_path = deepcopy([cities_lat, cities_lon, cities_name])
    best_path = deepcopy([cities_lat, cities_lon, cities_name])
    
    
    while (T > min_t):
        
        proposed_path = swap_city(randrange(0,29), randrange(0,29), cities_lat, cities_lon, cities_name)
        
        proposed_p, accepted_p = calculate_p(proposed_path,T), calculate_p(accepted_path,T)
        
        if proposed_p > accepted_p:
            
            accepted_path = proposed_path
            
            if get_loss(accepted_path) < get_loss(best_path):
                best_path = accepted_path

        else:
        
            U = uni(0,1)
            alpha = proposed_p/accepted_p

            if U <= alpha:
                accepted_path = proposed_path
        
        T = calculate_new_T(T,a)
        
        steps += 1
    
    return best_path

In [23]:
start = time()
best_path = city_SA(cities_info, 0.995, 1)
end = time()
print('Time taken: {} seconds'.format(end - start))

Time taken: 0.21273398399353027 seconds


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in double_scalars


In [17]:
best_path[2]

['г Москва',
 'г Санкт-Петербург',
 'г Ижевск',
 'г Екатеринбург',
 'г Нижний Новгород',
 'г Казань',
 'г Самара',
 'г Омск',
 'г Челябинск',
 'г Ростов-на-Дону',
 'г Уфа',
 'г Волгоград',
 'г Пермь',
 'г Красноярск',
 'г Воронеж',
 'г Саратов',
 'г Краснодар',
 'Самарская обл, г Тольятти',
 'Респ Ингушетия, г Сунжа',
 'г Барнаул',
 'г Новосибирск',
 'г Ульяновск',
 'г Владивосток',
 'г Ярославль',
 'г Иркутск',
 'Тюменская обл, г Тюмень',
 'г Махачкала',
 'г Хабаровск',
 'г Оренбург',
 'Кемеровская область - Кузбасс, г Новокузнецк']

### I didn't implement visualization, because it is too hard for me and I already spent 12 hours on this hw...